In [58]:
import gdown
import pandas as pd
import random
import json
from tqdm.auto import tqdm

In [59]:
tqdm.pandas()

In [60]:
# gdown.download('https://drive.google.com/file/d/17NBXx6l_6znStM52RJOKKKlxnHigavzq/view', fuzzy=True)

In [61]:
# tar -xvzf hackathon_objects.tar.gz

In [62]:
try: 

    goods_data = pd.read_parquet('../datasets/train_data.parquet')

    goods_pairs = pd.read_parquet('../datasets/train_pairs_w_target.parquet')
except FileNotFoundError:
    print('Using datasphere folder')
    
    goods_data = pd.read_parquet('../../files/datasets/train_data.parquet')

    goods_pairs = pd.read_parquet('../../files/datasets/train_pairs_w_target.parquet')

Using datasphere folder


In [63]:
goods_pairs.head()

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289
2,1.0,53602615,587809782
3,1.0,53888651,89598677
4,0.0,56930698,551526166


In [64]:
goods_data.head()

,variantid,characteristic_attributes_mapping,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64
0,51195767,"{""85"":[""Партнер-Электро""],""8229"":[""Удлинитель ...","Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
1,51196903,"{""85"":[""TDM Electric""],""8229"":[""Удлинитель быт...",Колодка TDM Electric пятиместная без заземлени...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],"[[0.42044494, -0.33794826, -0.037247024, 0.165...","[[0.16211876, -0.4455993, 0.6131705, 0.5954206...","[-0.48503304, 0.6264443, 0.6406273, -0.4696772..."
2,52061880,"{""85"":[""MINAMOTO""],""8229"":[""Батарейка""],""5111""...",Батарейка литиевая CR 1/3N 3V (CR11108) Minamo...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.66729844, -0.023996592, 0.42721167, 0.0205...","[[-0.8638098, -0.1625915, -0.034600798, 0.1337...","[-0.15832633, 0.44736174, 0.31883216, -0.55559..."
3,52067481,"{""85"":[""Duracell""],""8229"":[""Батарейка""],""5111""...","Батарейки DURACELL Ultra Power AA (LR6), 4 шт","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.3516068, 0.6306597, 0.80962396, -0.0104141...","[[-0.26286322, 0.22858405, 0.4346232, 0.023486...","[-0.42703134, 0.16898727, 0.53922343, -0.53523..."
4,52610752,"{""85"":[""HP""],""8229"":[""Картридж""],""5708"":[""для ...",Картридж лазерный HP 12A Q2612A черный для LJ ...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...","[черный, чер]","[[-0.009966308, 0.70965785, 0.10790472, -0.411...","[[0.09032486, 0.74205226, 0.30355096, -0.04755...","[-0.4892143, 0.47568643, 0.6505941, -0.4106509..."


In [65]:
a = eval(goods_data['characteristic_attributes_mapping'][0])

In [66]:
def dict_to_str(x):
    ans = ''
    for k in x.keys():
        ans += f'{k}: '
        try:
            for i, elem in enumerate(x[k]):
                if i != (len(x[k]) - 1):
                    ans += f'{elem}, '
                else:
                    ans += f'{elem}. '
        except ValueError:
            print(x[k])
            raise ValueError
            break
    return ans

def preprocess_dict(x):
    td = eval(x) # hehe
    td_new = dict(zip([f'[CAT_{k}]'  for k in td.keys()], td.values()))
    return dict_to_str(td_new)
    
preprocess_dict(goods_data['characteristic_attributes_mapping'][0])

'[CAT_85]: Партнер-Электро. [CAT_8229]: Удлинитель бытовой. [CAT_10096]: оранжевый. [CAT_5774]: Евровилка. [CAT_5739]: 220. [CAT_5776]: 10. [CAT_5778]: 10. [CAT_5777]: 2200. [CAT_5129]: Заземление. [CAT_5779]: IP44. [CAT_4383]: 1760. [CAT_7463]: 30. [CAT_4389]: Россия. '

In [67]:
goods_data['cat_proc'] = goods_data['characteristic_attributes_mapping'].progress_apply(preprocess_dict)

  0%|          | 0/457063 [00:00<?, ?it/s]

In [68]:
goods_data['to_nlp_data'] = goods_data['name'] + goods_data['cat_proc']

In [69]:
# apply tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [70]:
MAX_LENGTH = 128

def tokenize_function(examples):
    if type(examples) == str:
        examples = [examples]
    return tokenizer(
        examples, 
        return_tensors='pt',
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH
    )['input_ids']


In [71]:
orig_emb = tokenize_function('Каждый охотник желает охоты')
orig_emb.shape

torch.Size([1, 128])

In [72]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

In [73]:
to_nlp_dataloader = DataLoader(goods_data['to_nlp_data'], batch_size = 32, shuffle=False, drop_last=False)
embs = []
for elem in tqdm(to_nlp_dataloader):
    embs.append(tokenize_function(elem))

  0%|          | 0/14284 [00:00<?, ?it/s]

In [74]:
embs[0].shape

torch.Size([32, 128])

In [75]:
embs = torch.cat(embs)
embs.shape

torch.Size([457063, 128])

In [76]:
#goods_data['to_nlp_data_tokenized'] = goods_data['to_nlp_data'].progress_apply(tokenize_function)

In [77]:
'''
goods_data = goods_data.drop(
    [
        'pic_embeddings_resnet_v1',
        'to_nlp_data'.
        'cat_proc',
        'name',
        'main_pic_embeddings_resnet_v1'
    ]
)
'''

"\ngoods_data = goods_data.drop(\n    [\n        'pic_embeddings_resnet_v1',\n        'to_nlp_data'.\n        'cat_proc',\n        'name',\n        'main_pic_embeddings_resnet_v1'\n    ]\n)\n"

In [78]:
goods_data['normal_index'] = np.arange(goods_data.shape[0])
goods_data

,variantid,characteristic_attributes_mapping,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,cat_proc,to_nlp_data,normal_index
0,51195767,"{""85"":[""Партнер-Электро""],""8229"":[""Удлинитель ...","Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...",[CAT_85]: Партнер-Электро. [CAT_8229]: Удлинит...,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...",0
1,51196903,"{""85"":[""TDM Electric""],""8229"":[""Удлинитель быт...",Колодка TDM Electric пятиместная без заземлени...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],"[[0.42044494, -0.33794826, -0.037247024, 0.165...","[[0.16211876, -0.4455993, 0.6131705, 0.5954206...","[-0.48503304, 0.6264443, 0.6406273, -0.4696772...",[CAT_85]: TDM Electric. [CAT_8229]: Удлинитель...,Колодка TDM Electric пятиместная без заземлени...,1
2,52061880,"{""85"":[""MINAMOTO""],""8229"":[""Батарейка""],""5111""...",Батарейка литиевая CR 1/3N 3V (CR11108) Minamo...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.66729844, -0.023996592, 0.42721167, 0.0205...","[[-0.8638098, -0.1625915, -0.034600798, 0.1337...","[-0.15832633, 0.44736174, 0.31883216, -0.55559...",[CAT_85]: MINAMOTO. [CAT_8229]: Батарейка. [CA...,Батарейка литиевая CR 1/3N 3V (CR11108) Minamo...,2
3,52067481,"{""85"":[""Duracell""],""8229"":[""Батарейка""],""5111""...","Батарейки DURACELL Ultra Power AA (LR6), 4 шт","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.3516068, 0.6306597, 0.80962396, -0.0104141...","[[-0.26286322, 0.22858405, 0.4346232, 0.023486...","[-0.42703134, 0.16898727, 0.53922343, -0.53523...",[CAT_85]: Duracell. [CAT_8229]: Батарейка. [CA...,"Батарейки DURACELL Ultra Power AA (LR6), 4 шт[...",3
4,52610752,"{""85"":[""HP""],""8229"":[""Картридж""],""5708"":[""для ...",Картридж лазерный HP 12A Q2612A черный для LJ ...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...","[черный, чер]","[[-0.009966308, 0.70965785, 0.10790472, -0.411...","[[0.09032486, 0.74205226, 0.30355096, -0.04755...","[-0.4892143, 0.47568643, 0.6505941, -0.4106509...",[CAT_85]: HP. [CAT_8229]: Картридж. [CAT_5708]...,Картридж лазерный HP 12A Q2612A черный для LJ ...,4
...,...,...,...,...,...,...,...,...,...,...,...
457058,822394794,"{""85"":[""Poco""],""8229"":[""Смартфон""],""10096"":[""с...","Смартфон Poco M5s 4/128Gb Grey 4/128 ГБ, серый","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Смартфо...","[grey, серый]","[[0.5289667, -0.8587297, 0.6250174, 0.10048234...","[[0.48594046, -0.8139478, -0.11380915, -0.3833...","[-0.56425023, 0.41288334, 0.5501014, -0.330675...",[CAT_85]: Poco. [CAT_8229]: Смартфон. [CAT_100...,"Смартфон Poco M5s 4/128Gb Grey 4/128 ГБ, серый...",457058
457059,822409141,"{""85"":[""vlp""],""8229"":[""Чехол для смартфона""],""...",Чехол vlp Silicone с MagSafe для iPhone 14 Plu...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Чехол"",...","[black, черный]","[[-0.25175428, -0.5215634, 0.88485634, 0.72421...","[[0.22031438, -1.3273709, 0.80755866, 0.343482...","[-0.073415995, 0.39561644, 0.4324598, -0.63042...",[CAT_85]: vlp. [CAT_8229]: Чехол для смартфона...,Чехол vlp Silicone с MagSafe для iPhone 14 Plu...,457059
457060,822467917,"{""85"":[""Acer""],""10096"":[""зеленый""],""4460"":[""От...","Ноутбук Acer Aspire A317-32-C3M5 (17.3""/Intel...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Компьют...","[black, черный, зеленый]",None,"[[0.7038868, -0.2959186, -0.7599652, -0.354495...","[-0.631185, 0.3781632, 0.50562716, -0.65348136...",[CAT_85]: Acer. [CAT_10096]: зеленый. [CAT_446...,"Ноутбук Acer Aspire A317-32-C3M5 (17.3""/Intel...",457060
457061,822735361,"{""85"":[""Syncwire""],""8229"":[""Видео-кабель""],""10...","Кабель для видеонаблюдения (КВК) 2П 2х0,5 LV 1...","{""1"": ""EPG"", ""2"": ""Электроника"", 

#  dataset

In [79]:
def symm_pairs(df):
    # todo: complete cliques of goods
    
    df2 = df.copy()
    cols = ["variantid1", "variantid2"]
    for c in cols:
        if c not in df2.columns:
            raise UserWarning
    df2 = df2.rename(columns={cols[0]: cols[1], cols[1]: cols[0]})
    return pd.concat([df, df2]).drop_duplicates()

In [80]:
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader


class ContrastiveDataset(Dataset):
    def __init__(self, original_dataset, pairs_dataset, embedding_columns, data):
        self.original_dataset = original_dataset.set_index('variantid')
        self.pairs_dataset = pairs_dataset
        self.embedding_columns = embedding_columns
        self.dataset = None
        self.data = data
        self.prepare_dataset()
        

    def prepare_dataset(
        self,
    ):
        neg_pairs = (
            self.pairs_dataset[self.pairs_dataset.target == 0.0]
            .groupby("variantid1")
            .variantid2.apply(lambda x: list(x))
            .to_frame()
        )
        pos_pairs = (
            self.pairs_dataset[self.pairs_dataset.target == 1.0]
            .groupby("variantid1")
            .variantid2.apply(lambda x: list(x))
            .to_frame()
        )
        self.dataset = pos_pairs.merge(
            neg_pairs,
            how="outer",
            left_index=True,
            right_index=True,
            indicator=True,
            suffixes=["_pos", "_neg"],
        )
        sol = pd.Series(
            np.where(
                self.dataset.variantid2_pos.notnull(),
                self.dataset.variantid2_pos,
                pd.Series(self.dataset.index).apply(lambda x: [x]),
            )
        )
        self.dataset["variantid2_pos"] = sol.set_axis(self.dataset.index)
        ld = pd.Series(self.dataset.index).sample(frac=1).apply(lambda x: [x])
        sol = pd.Series(
            np.where(self.dataset.variantid2_neg.notnull(), self.dataset.variantid2_neg, ld)
        )
        self.dataset["variantid2_neg"] = sol.set_axis(self.dataset.index)
        self.dataset = self.dataset.reset_index()


    def __getitem__(self, index):
        original_idx = self.dataset.loc[index].variantid1
        positive_idx = self.dataset.loc[index].variantid2_pos
        negative_idx = self.dataset.loc[index].variantid2_neg
        if len(positive_idx)>1:
            val = np.random.choice(len(positive_idx), 1)[0]
            positive_idx = positive_idx[val]
        else:
            positive_idx = positive_idx[0]
        if len(negative_idx)>1:
            val = np.random.choice(len(negative_idx), 1)[0]
            negative_idx = negative_idx[val]
        else:
            negative_idx = negative_idx[0]
        
        original_data = self.data[self.original_dataset.loc[original_idx]['normal_index']]
        positive_pair = self.data[self.original_dataset.loc[positive_idx]['normal_index']]
        negative_pair = self.data[self.original_dataset.loc[negative_idx]['normal_index']]
        
        '''
        original_data = self.original_dataset.loc[original_idx]['to_nlp_data_tokenized']
        #torch.cat([torch.tensor(self.original_dataset.loc[original_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[original_idx]['name_bert_64'])], axis = 0)
        positive_pair = self.original_dataset.loc[positive_idx]['to_nlp_data_tokenized']
        #torch.cat([torch.tensor(self.original_dataset.loc[positive_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[positive_idx]['name_bert_64'])], axis =0)
        negative_pair = self.original_dataset.loc[negative_idx]['to_nlp_data_tokenized']
        #torch.cat([torch.tensor(self.original_dataset.loc[negative_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[negative_idx]['name_bert_64'])], axis = 0)
        '''
        
        return original_data, positive_pair, negative_pair, original_idx

    def __len__(self):
        return len(self.dataset)


# Actual data

In [81]:
symm_goods_pairs = symm_pairs(goods_pairs)

In [82]:
pos_pairs = symm_goods_pairs[symm_goods_pairs.target ==1].groupby('variantid1').variantid2.apply(lambda x: list(x)).to_frame()

In [83]:
#checking if lists contain duplicates
pos_pairs.variantid2.apply(lambda x: len(x) !=len(set(x))).value_counts()

variantid2
False    216911
Name: count, dtype: int64

In [84]:
# добавить все симметричные positive pairs
pos_pairs.variantid2.str.len().value_counts()

variantid2
1     189650
2      18977
3       3806
4       1403
5        734
6        561
7        380
8        287
9        247
10       190
11       166
12       129
13       115
14        72
15        55
16        40
17        24
18        22
20        15
19        14
21         6
22         5
25         3
26         3
23         2
27         2
24         2
28         1
Name: count, dtype: int64

In [85]:
embedding_columns = ['main_pic_embeddings_resnet_v1', 'name_bert_64']

contrastive_dataset = ContrastiveDataset(goods_data, symm_goods_pairs, embedding_columns, embs)

In [86]:
contrastive_dataset[0][0].shape

torch.Size([128])

In [95]:
contrastive_dataloader = DataLoader(contrastive_dataset, batch_size=2, shuffle=True, drop_last= True)
next(iter(contrastive_dataloader))[0].shape

torch.Size([2, 128])

# Basic train example

In [ ]:
contrastive_dataset.dataset.shape

In [89]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

In [90]:
#!g1.1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [91]:
#!g1.1

def triplet_loss(anchor, positive, negative, margin=1.0):
    distance_positive = torch.norm(anchor - positive, dim=1)
    distance_negative = torch.norm(anchor - negative, dim=1)
    loss = torch.clamp(distance_positive - distance_negative + margin, min=0.0).mean()
    return loss

# Example parameters
input_size = 192
hidden_size = 64
embedding_size = 32
learning_rate = 0.001
num_epochs = 10

# Create an instance of the model
# model = ContrastiveModel(input_size, hidden_size, embedding_size).to(device)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", 
    num_labels=embedding_size
).to(device)
model.train()

# Define the optimizer and the loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [92]:
#!g1.1
'''
with torch.no_grad():
    orig, pos, neg, idx = next(iter(contrastive_dataloader))

    ans = model(orig.to(device))
    ans
'''

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/dtypes.py:585: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,
Failed to deserialize variable 'ans'. Run the following code to delete it:
  del_datasphere_variables('ans')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "<string>", line 7, in __init__
  File "/home/jupyter/.local/lib/python3.8/site-packages/transformers/utils/generic.py", line 246, in __post_init__
    if other_fields_are_none and not is_tensor(first_field):
  File "/home/jupyter/.local/lib/python3.8/site-packages/transformers/utils/generic.py", line 86, in is_tensor
    import tensorflow as tf
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/usr/lo

In [96]:
#!g1.1
import gc
torch.cuda.empty_cache()
gc.collect()

3

In [ ]:
#!g1.1

# Train loop
for epoch in tqdm(range(num_epochs)):
    total_loss = 0.0
    
    for original_data, positive_pair, negative_pair, _ in tqdm(contrastive_dataloader):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        anchor = model(original_data.to(device)).logits
        positive = model(positive_pair.to(device)).logits
        negative = model(negative_pair.to(device)).logits
        
        # Compute the triplet loss
        loss = triplet_loss(anchor, positive, negative)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
   #     print(loss.item())
        total_loss += loss.item()
    
    average_loss = total_loss / len(contrastive_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/228370 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'contrastive_ozon.pt')

In [ ]:
def predict(model, dataloader):
    y_idxs = []
    y_embed = []
    for orig, _, _, idx in tqdm(dataloader):
        embed = model(orig.to(device))
        y_idxs.extend([i.detach().cpu().item() for i in idx])
        y_embed.extend([[j.item() for j in i.detach().cpu()] for i in embed])
    return y_idxs, y_embed

y_idxs, y_embed = predict(model, contrastive_dataloader)       


In [ ]:
embed = pd.DataFrame({'variant': y_idxs, 'embed': y_embed})

In [ ]:
classif = goods_pairs.merge(embed, left_on = 'variantid1', right_on = 'variant', how = 'left', suffixes = ['', '_1'])

In [ ]:
classif = classif.merge(embed, left_on = 'variantid2', right_on='variant', how = 'left', suffixes = ['', '_2'])

In [ ]:
classif.drop(['variant', 'variant_2'],axis = 1, inplace = True)

In [ ]:
classif

# Catboost ingerence on cos sim

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_pic_features(main_pic_embeddings_1,
                     main_pic_embeddings_2,
                     percentiles):
    """Calculate distances percentiles for 
    pairwise pic distances. Percentiles are useful 
    when product has several pictures.
    """
    
    if main_pic_embeddings_1 is not None and main_pic_embeddings_2 is not None:     
        dist_m = cosine_similarity(
            np.array(main_pic_embeddings_1)[None, ...], np.array(main_pic_embeddings_2)[None, ...]
        )
    else:
        dist_m = np.array([[-1]])

    pair_features = []
    pair_features += np.percentile(dist_m, percentiles).tolist()

    return pair_features

In [ ]:
from functools import partial
classif = classif.dropna()
get_pic_features_func = partial(
    get_pic_features,
    percentiles=[0, 25, 50]
)
classif[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    classif[["embed", "embed_2"]].apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1
    )
)

In [ ]:
classif

In [ ]:
classif = classif.merge(goods_data[['variantid', 'categories']], left_on = 'variantid1', right_on = 'variantid', how = 'left')

In [ ]:
classif["cat3"] = classif["categories"].apply(lambda x: json.loads(x)["3"])

In [ ]:
cat3_counts = classif["cat3"].value_counts().to_dict()

In [ ]:
classif["cat3_grouped"] = classif["cat3"].apply(lambda x: x if cat3_counts[x] > 1000 else "rest")

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(
    classif[['target', "variantid1", "variantid2", 'pic_dist_0_perc', 'cat3_grouped']], 
    test_size=0.1, random_state=42, stratify=classif[["target", "cat3_grouped"]]
)

y_test = X_test[["target", "variantid1", "variantid2" ]]
X_test = X_test.drop(["target"], axis=1)

y_train = X_train["target"]

X_train = X_train.drop(["target", "variantid1", "variantid2"], axis=1)

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
model = CatBoostClassifier()

model.fit(
    X_train[['pic_dist_0_perc']], y_train,
    plot=True,
    verbose=True,
    use_best_model=True,
    early_stopping_rounds=50,
    metric_period=100
)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    target_df: pd.DataFrame,
    predictions_df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:
    
    df = target_df.merge(predictions_df, on=["variantid1", "variantid2"])
    
    y_true = df["target"]
    y_pred = df["scores"]
    categories = df[cat_column]
    
    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    for i, category in enumerate(unique_cats):
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        y, x, thr = precision_recall_curve(y_true_cat, y_pred_cat)
        gt_prec_level_idx = np.where(y >= prec_level)[0]

        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
                weights.append(counts[i] / len(categories))
        except ValueError as err:
            pr_aucs.append(0)
            weights.append(0)
    return np.average(pr_aucs, weights=weights)

In [ ]:
X_test["scores"] = model.predict_proba(X_test[['pic_dist_0_perc']])[:, 1]

In [ ]:
pr_auc_macro_metr = pr_auc_macro(
    target_df=y_test, 
    predictions_df=X_test,
    prec_level=0.75,
    cat_column="cat3_grouped"
)

pr_auc_macro_metr

In [ ]:
#используется только одна картинка, можно делать positive pairs со всеми

In [ ]:
X_test["scores"].hist()

In [ ]:
#!c1.4
